In [1]:
# INSTALL REQUIRED PACKAGES
!pip install qiskit qiskit-aer --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [2]:
# IMPORT LIBRARIES
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np


In [3]:
# QFT IMPLEMENTATION
def qft(n):
    qc = QuantumCircuit(n)

    for i in range(n):
        qc.h(i)
        for j in range(i+1, n):
            qc.cp(np.pi / (2 ** (j - i)), j, i)

    qc.barrier()

    for i in range(n // 2):
        qc.swap(i, n - i - 1)

    return qc


In [4]:
# IQFT IMPLEMENTATION — WITH NO BARRIER (GATE SAFE)
def iqft_gate_safe(n):
    qc = QuantumCircuit(n)

    # Swap qubits
    for i in range(n // 2):
        qc.swap(i, n - i - 1)

    # Inverse rotations + Hadamards
    for i in reversed(range(n)):
        for j in reversed(range(i+1, n)):
            qc.cp(-np.pi / (2 ** (j - i)), j, i)
        qc.h(i)

    return qc


In [5]:
# IQFT FOR DISPLAY PURPOSES
def iqft_display(n):
    qc = iqft_gate_safe(n)
    qc.barrier()
    return qc


In [6]:
# RUN FUNCTION (REPLACES execute)
def run(qc, shots=1024):
    backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    return job.result().get_counts()


In [7]:
print("=== TASK 1: QFT for 2, 3, and 4 Qubits ===")

for n in [2, 3, 4]:
    print(f"\nQFT Circuit for {n} Qubits:")
    print(qft(n).draw('text'))


=== TASK 1: QFT for 2, 3, and 4 Qubits ===

QFT Circuit for 2 Qubits:
     ┌───┐               ░    
q_0: ┤ H ├─■─────────────░──X─
     └───┘ │P(π/2) ┌───┐ ░  │ 
q_1: ──────■───────┤ H ├─░──X─
                   └───┘ ░    

QFT Circuit for 3 Qubits:
     ┌───┐                                      ░    
q_0: ┤ H ├─■────────■───────────────────────────░──X─
     └───┘ │P(π/2)  │       ┌───┐               ░  │ 
q_1: ──────■────────┼───────┤ H ├─■─────────────░──┼─
                    │P(π/4) └───┘ │P(π/2) ┌───┐ ░  │ 
q_2: ───────────────■─────────────■───────┤ H ├─░──X─
                                          └───┘ ░    

QFT Circuit for 4 Qubits:
     ┌───┐                                                                     »
q_0: ┤ H ├─■────────■─────────────■────────────────────────────────────────────»
     └───┘ │P(π/2)  │       ┌───┐ │                                            »
q_1: ──────■────────┼───────┤ H ├─┼────────■────────■──────────────────────────»
                   

In [8]:
print("\n=== TASK 2: Inverse QFT (IQFT) for 3 Qubits ===")
print(iqft_display(3).draw('text'))



=== TASK 2: Inverse QFT (IQFT) for 3 Qubits ===
                                                ┌───┐ ░ 
q_0: ─X──────────────────────■─────────■────────┤ H ├─░─
      │                ┌───┐ │         │P(-π/2) └───┘ ░ 
q_1: ─┼───────■────────┤ H ├─┼─────────■──────────────░─
      │ ┌───┐ │P(-π/2) └───┘ │P(-π/4)                 ░ 
q_2: ─X─┤ H ├─■──────────────■────────────────────────░─
        └───┘                                         ░ 


In [9]:
# PHASE ESTIMATION — FIXED VERSION

def phase_estimation(phi, t=3):
    qc = QuantumCircuit(t + 1, t)

    qc.h(range(t))

    # Controlled unitary operations
    for k in range(t):
        qc.cp(2 * np.pi * phi * (2 ** k), k, t)

    # Use SAFE IQFT gate (NO barrier)
    iqft_gate = iqft_gate_safe(t).to_gate(label="IQFT")
    qc.append(iqft_gate, range(t))

    qc.measure(range(t), range(t))

    return qc


phi = 0.125
qc_pe = phase_estimation(phi, t=3)

print("\n=== TASK 3: Phase Estimation Circuit ===")
print(qc_pe.draw('text'))

counts = run(qc_pe)
print("\nPhase Estimation Output:", counts)



=== TASK 3: Phase Estimation Circuit ===
     ┌───┐                         ┌───────┐┌─┐      
q_0: ┤ H ├─■───────────────────────┤0      ├┤M├──────
     ├───┤ │                       │       │└╥┘┌─┐   
q_1: ┤ H ├─┼────────■──────────────┤1 IQFT ├─╫─┤M├───
     ├───┤ │        │              │       │ ║ └╥┘┌─┐
q_2: ┤ H ├─┼────────┼────────■─────┤2      ├─╫──╫─┤M├
     └───┘ │P(π/4)  │P(π/2)  │P(π) └───────┘ ║  ║ └╥┘
q_3: ──────■────────■────────■───────────────╫──╫──╫─
                                             ║  ║  ║ 
c: 3/════════════════════════════════════════╩══╩══╩═
                                             0  1  2 

Phase Estimation Output: {'000': 1024}


In [10]:
print("\n=== TASK 4: QFT Measurement Example ===")

qc_meas = qft(3)
qc_meas.measure_all()

counts_qft = run(qc_meas)
print("QFT Measurement Results:", counts_qft)



=== TASK 4: QFT Measurement Example ===
QFT Measurement Results: {'111': 119, '011': 136, '101': 117, '010': 142, '000': 136, '001': 128, '110': 118, '100': 128}


In [11]:
print("\n=== TASK 5: Circuit Visualization Example ===")
print(qft(3).draw('text'))



=== TASK 5: Circuit Visualization Example ===
     ┌───┐                                      ░    
q_0: ┤ H ├─■────────■───────────────────────────░──X─
     └───┘ │P(π/2)  │       ┌───┐               ░  │ 
q_1: ──────■────────┼───────┤ H ├─■─────────────░──┼─
                    │P(π/4) └───┘ │P(π/2) ┌───┐ ░  │ 
q_2: ───────────────■─────────────■───────┤ H ├─░──X─
                                          └───┘ ░    
